In [ ]:
import sqlite3

# ## Open enhance_google_places.db
# with sqlite3.connect('result\enhanced_google_places.db') as conn:
#     cursor = conn.cursor()
    
#     cursor.execute("SELECT DISTINCT categories_0 from places")
#     categories = cursor.fetchall()
#     print(categories)

conn = sqlite3.connect('result\enhanced_google_places.db')
cursor = conn.cursor()

## 4 columns
cursor.execute("SELECT DISTINCT categories_0, categoryname, cuisine_tags, primary_cuisine from places")
categories = cursor.fetchall()

cater_bags = set()
for a, b, c, d in categories:
    cater_list_as_string = str(a) + ',' + str(b) + ',' + str(c) + ',' + str(d)
    cater_list = cater_list_as_string.split(',')
    cater_list = [cater.strip() for cater in cater_list]
    for cater in cater_list:
        cater_bags.add(cater)

with open('result\cater_bags.txt', 'w', encoding='utf-8') as f:
    for a in cater_bags:
        f.write(a)
        f.write('\n')


In [ ]:
print(cater_bags)

In [ ]:
CATEGORIES = ["accommodation", "activity", "catering",
        "commercial", "entertainment",
        "heritage", "leisure",
        "manufacturing", "natural",
        "religion", "rental", 
        "service", "tourism"]

In [ ]:
## format return: "cater : binarymask of length 13"
def binary_to_cater(binary_mask):
    return [CATEGORIES[i] for i in range(len(binary_mask)) if binary_mask[i] == '1']

def binary_string_to_num(binary_string):
    return int(binary_string, 2)


In [ ]:
   
cater_to_bin = {}
with open('result/auto_categorize.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line == '':
            continue
        cater, bin_mask = line.split(':')
        cater, bin_mask = cater.strip(), bin_mask.strip()
        print(bin_mask)
        assert len(bin_mask) == 13
        cater_to_bin[cater] = bin_mask



In [ ]:
def process_catergories(catergories):
    ## categories to binary mask
    mask_list = []
    for a, b, c, d in catergories:
        cater_list_as_string = str(a) + ',' + str(b) + ',' + str(c) + ',' + str(d)
        cater_list = cater_list_as_string.split(',')
        cater_list = [cater.strip() for cater in cater_list]
        ## operation or on all of binary_to_cater[cater_list]
        bin_mask = [binary_string_to_num(cater_to_bin.get(cater, '0'*13)) for cater in cater_list]
        result = 0
        for num in bin_mask:
            result = result | num
        mask_list.append(bin(result)[2:].zfill(13))
    return mask_list









In [ ]:

conn = sqlite3.connect('result\enhanced_google_places.db')
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM places")
total_places = cursor.fetchone()[0]
print(total_places)
cursor.execute("SELECT categories_0, categoryname, cuisine_tags, primary_cuisine from places")
categories = cursor.fetchall()
print(len(categories))
mask_list = process_catergories(categories)
mask_list = [list(binary_string) for binary_string in mask_list]
print(mask_list[:50])


In [ ]:
for cat in CATEGORIES:
    cat_name = f'cat_{cat}'
    cursor.execute(f'ALTER TABLE places ADD COLUMN "{cat_name}" INTEGER DEFAULT 0')

conn.commit()



In [ ]:
cursor.execute("SELECT rowid FROM places ORDER BY rowid ASC")
rows = cursor.fetchall()
ids = [r[0] for r in rows]

assert len(ids) == len(mask_list)
print(ids[:50])

In [ ]:
for i, cat in enumerate(CATEGORIES):
    cat_name = f'cat_{cat}'
    for j, place_id in enumerate(ids):
        bit = int(mask_list[j][i])  # '0'/'1' -> 0/1
        cursor.execute(
            f'UPDATE places SET "{cat_name}" = ? WHERE rowid = ?',
            (bit, place_id)
        )

conn.commit()

In [ ]:
## analyze num of each cat
for cat in CATEGORIES:
    cat_name = f'cat_{cat}'
    cursor.execute(f'SELECT COUNT(*) FROM places WHERE {cat_name} = 1')
    num = cursor.fetchall()
    print(cat, num)

In [ ]:
all_zeros = ' AND '.join([f'cat_{cat} = 0' for cat in CATEGORIES])
print(all_zeros)  # kiểm tra: cat_accommodation = 0 AND cat_activity = 0 AND ...

cursor.execute(f'SELECT COUNT(*) FROM places WHERE {all_zeros}')
count = cursor.fetchone()[0]
print(count)